In [1]:
import numpy as np

In [ ]:
projectors = [np.array([[1, 0], [0, 0]]), np.array([[0, 0], [0, 1]])]

class Reg:
    def __init__(self, n):
        self.n = n
        self.psi = np.zeros((2,)*n)
        self.psi[(0,)*n] = 1

H_matrix = 1/np.sqrt(2)*np.array([[1,1],
                                  [1,-1]])

def H(i, reg):
    reg.psi = np.tensordot(H_matrix, reg.psi, (1,i))
    reg.psi = np.moveaxis(reg.psi,0,i)


def project(i, j, reg):
    projected = np.tensordot(projectors[j], reg.psi, (1,i))
    return np.moveaxis(projected, 0, i)


from scipy.linalg import norm

def measure(i, reg):
    projected = project(i, 0, reg)
    norm_projected = norm(projected.flatten())
    if np.random.random() < norm_projected**2:
        reg.psi = projected/norm_projected
        return 0
    else:
        projected = project(i,1,reg)
        reg.psi = projected/norm(projected)
        return 1

Example on applying a Hadamard gate to the 0th qubit of the all zero state

In [9]:
for i in range(100):
    reg = Reg(4)
    H(0, reg)
    print(measure(0, reg), end='')

1010011101101101010111101011011111110100110100100100111110111100010110111101000011011001001011001010

Example case when get the outcome but then do not reinitialize the state after measurement

In [12]:
reg = Reg(4)
H(0, reg)
for i in range(100):
    print(measure(0,reg), end='')

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

generate Bell state, and then measure the first qubit

In [16]:
def CNOT(control, target, reg):
    reg.psi = np.tensordot(np.reshape(np.array([[1,0,0,0],
                                                [0,1,0,0],
                                                [0,0,0,1],
                                                [0,0,1,0]]), (2,2,2,2)), reg.psi, ((2,3), (control, target)))

    reg.psi = np.moveaxis(reg.psi, (0,1), (control, target))

def generate_GHZ(reg):
    H(0, reg)
    for i in range(reg.n-1):
        CNOT(i, i+1, reg)

for i in range(100):
    reg = Reg(4)
    generate_GHZ(reg)
    print(measure(0,reg),end='')
    print(measure(1,reg),end=' ')

11 11 11 11 11 11 11 11 00 00 11 11 00 11 11 00 00 11 00 11 11 11 11 00 11 11 00 11 00 00 11 11 00 11 11 00 11 00 11 11 00 00 11 11 11 11 11 00 00 11 11 00 11 11 11 11 00 00 00 11 00 11 00 11 00 00 11 00 11 00 11 11 00 00 11 11 11 00 11 11 00 11 00 00 11 00 11 11 11 11 11 11 11 00 00 00 00 00 11 00 

In [ ]:
import time

H_gate = 1/np.sqrt(2)*np.array([[1,1],
                    [1,-1]])

X_gate = np.array([[0,1],
                   [1,0]])

CNOT_gate = np.array([[1,0,0,0],
                    [0,1,0,0],
                    [0,0,0,1],
                    [0,0,1,0]]).reshape(2,2,2,2)

def apply_single_qubit_gate(state, gate, target_qubit):

    psi_updated = np.tensordot(gate, state, axes=(1,target_qubit))
    psi_updated = np.moveaxis(psi_updated, 0, target_qubit)

    return psi_updated

def apply_cnot_gate(state, target_qubit, control_qubit):

    psi_updated = np.tensordot(CNOT_gate, state, ((2,3), (control_qubit, target_qubit)))
    psi_updated = np.moveaxis(state, (0,1), (control_qubit, target_qubit))

    return psi_updated

def quantum_simulator(N):
    
    start_time = time.time()

    psi = np.zeros((2,)*N)
    psi[(0,)*N] = 1 #initiate example state: |000...001> 
    
    for i in range(N): # apply the gate for each qubit
        psi = apply_single_qubit_gate(psi, X_gate, i)
        psi = apply_single_qubit_gate(psi, H_gate, i)
    
    psi = apply_cnot_gate(psi, N-1, 0)
    
    end_time = time.time()
    runtime = end_time - start_time

    result = psi
    
    return result, runtime


In [ ]:
def measure(qubit, state):
    projected = np.tensordot(projectors[0], state, (1,qubit))
    print(projected)
    norm_projected = norm(projected.flatten())
    print(norm_projected)

    if np.random.random() < norm_projected**2:
        psi = projected/norm_projected
        return 0
    else:
        projected = np.tenso

psi = np.zeros((2,)*2)
psi[0,0] = 1

measure(qubit=0, state=psi)

[[1. 0.]
 [0. 0.]]
1.0


In [56]:
import numpy as np
from scipy.linalg import norm

# Define projectors for |0⟩ and |1⟩
projectors = [np.array([[1, 0], [0, 0]]), np.array([[0, 0], [0, 1]])]

# Define Hadamard matrix
H_matrix = 1/np.sqrt(2) * np.array([[1, 1],
                                    [1, -1]])

# Initialize the state
def initialize_state(n):
    psi = np.zeros((2,) * n)
    psi[(0,) * n] = 1  # Start in the |0...0⟩ state
    return psi

# Apply Hadamard gate to qubit i
def H(i, psi):
    psi = np.tensordot(H_matrix, psi, (1, i))
    psi = np.moveaxis(psi, 0, i)
    return psi

# Project state onto |0⟩ or |1⟩ for qubit i
def project(i, j, psi):
    projected = np.tensordot(projectors[j], psi, (1, i))
    return np.moveaxis(projected, 0, i)

# Measure qubit i
def measure(i, psi):
    # Project onto |0⟩ for qubit i and calculate the probability of measuring |0⟩
    projected_0 = project(i, 0, psi)
    prob_0 = norm(projected_0.flatten())**2  # Probability of |0⟩

    # Decide the measurement outcome based on probability
    if np.random.random() < prob_0:
        psi = projected_0 / norm(projected_0)  # Collapse to |0⟩ state for qubit i
        return 0, psi
    else:
        projected_1 = project(i, 1, psi)
        psi = projected_1 / norm(projected_1)  # Collapse to |1⟩ state for qubit i
        return 1, psi

# Example Usage
n_qubits = 2
psi = initialize_state(n_qubits)  # Initialize a 2-qubit state in |00⟩

# Apply Hadamard gate to both qubits to create superposition
for i in range(n_qubits):
    psi = H(i, psi)

# Measure the first qubit
outcome, psi = measure(0, psi)
print("Measurement outcome of qubit 0:", outcome)
print("State after measurement:", psi)


Measurement outcome of qubit 0: 1
State after measurement: [[0.         0.        ]
 [0.70710678 0.70710678]]
